## Data preprocessing

In [5]:
import movekit as mkit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Read data input

In [7]:
# Enter path to CSV file
path = "./datasets/fish-5.csv"

# Alternative: enter path to Excel file
# path = "./datasets/fish-5.xlsx"

In [8]:
# Read in file using 
data = mkit.read_data(path)
data.head()

,time,animal_id,x,y
0,1,312,405.29,417.76
1,1,511,369.99,428.78
2,1,607,390.33,405.89
3,1,811,445.15,411.94
4,1,905,366.06,451.76


In [9]:
# Simple call of the preprocessing method
preprocessed_data = mkit.preprocess(data)

Total number of missing values =  0
time         0
animal_id    0
x            0
y            0
dtype: int64


In [10]:
# OPTIONAL: more parameters to control the preprocessing of data

# preprocessed_data = mkit.preprocess(data, dropna=True, interpolation=False, limit=1, limit_direction="forward", inplace=False, method="linear")

# Paramters 
#  data: DataFrame to perform preprocessing on
#  dropna: Optional parameter to drop columns with  missing values for 'time' and 'animal_id'
#  interpolate: Optional parameter to perform linear interpolation
#  limit: Maximum number of consecutive NANs to fill
#  limit_direction: If limit is specified, consecutive NaNs will be filled in this direction.
#  method: Interpolation technique to use. Default is "linear".
#  order: To be used in case of polynomial interpolation.

In [11]:
# OPTIONAL: converting positional data into scale, defined by user
# preprocessed_data = mkit.convert_measueres(preprocessed_data, x_min = 0, x_max = 100, y_min = 0, y_max = 100)

In [12]:
# save cleaned features to csv 
preprocessed_data.to_csv("datasets/fish-5-cleaned.csv", index=False)